In [1]:
import os
import shutil
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
#import tensorflow_hub as hub
#import tensorflow_text as text

print("TF Version: ", tf.__version__)

c:\Users\mgf-l\Documents\University\GitHub\fb_event_scraper\cat_pred\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TF Version:  2.16.1


In [2]:
import matplotlib.pyplot as plt


def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

data_loc = "C:\\Users\\mgf-l\\Desktop\\structured_data.xlsx"
df = pd.read_excel(data_loc)
    
df = df.dropna(subset=['category', 'title'])


df = df[['category', 'title']]


X = df['title']   
y = df['category']  


train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42) 

train_dataset = tf.data.Dataset.from_tensor_slices((train_X, train_y))
test_dataset = tf.data.Dataset.from_tensor_slices((test_X, test_y))
train_dataset


<_TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))>

In [4]:
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

text:  b'Pub kronan'
label:  b'Pub'


In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define tokenizer
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_X)  # Fit only to training data

# Convert texts to sequences
train_sequences = tokenizer.texts_to_sequences(train_X)
test_sequences = tokenizer.texts_to_sequences(test_X)

# Pad sequences
max_length = max(len(x) for x in train_sequences)  # You can set a specific max_length if you prefer
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post')


In [11]:
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

encoder = LabelEncoder()
encoder.fit(train_y)  # Fit encoder on the training labels

# Transform categories to integers
train_labels = encoder.transform(train_y)
test_labels = encoder.transform(test_y)

# Convert to one-hot encoding
train_labels_one_hot = tf.keras.utils.to_categorical(train_labels)
test_labels_one_hot = tf.keras.utils.to_categorical(test_labels)


In [15]:
scrape_df = pd.read_excel("C:\\Users\\mgf-l\\Desktop\\structured_data.xlsx", sheet_name = "scraped")

TypeError: read_excel() got an unexpected keyword argument 'sheet'

In [7]:
for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])

texts:  b'Pub'

labels:  b'Pub'


In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=16, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(len(encoder.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



c:\Users\mgf-l\Documents\University\GitHub\fb_event_scraper\cat_pred\Lib\site-packages\keras\src\layers\core\embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [13]:
history = model.fit(train_padded, train_labels_one_hot, epochs=30, validation_data=(test_padded, test_labels_one_hot))


Epoch 1/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.2708 - loss: 2.3648 - val_accuracy: 0.2626 - val_loss: 2.2327
Epoch 2/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2703 - loss: 2.1692 - val_accuracy: 0.2626 - val_loss: 2.0332
Epoch 3/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2886 - loss: 1.9761 - val_accuracy: 0.2626 - val_loss: 1.9577
Epoch 4/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2775 - loss: 1.9697 - val_accuracy: 0.2997 - val_loss: 1.8958
Epoch 5/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3095 - loss: 1.9111 - val_accuracy: 0.3401 - val_loss: 1.8268
Epoch 6/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3650 - loss: 1.8253 - val_accuracy: 0.3535 - val_loss: 1.7394
Epoch 7/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3937 - loss: 1.7108 - val_accuracy: 0.5051 - val_loss: 1.6371
Epoch 8/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4941 - loss: 1.6350 - val_accuracy: 0.5354 - val_loss:

In [14]:
# Evaluate the model
loss, accuracy = model.evaluate(test_padded, test_labels_one_hot)
print(f"Test accuracy: {accuracy}")



# Making predictions
predictions = model.predict(test_padded)
predicted_categories = encoder.inverse_transform([np.argmax(p) for p in predictions])


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9256 - loss: 0.2971 
Test accuracy: 0.9292929172515869
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
